In [5]:
import pennylane as qml
from pennylane import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import cv2
import random
import os
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import random
from PIL import Image
from scipy.optimize import minimize
import glob

In [6]:
def get_category_dict(dataset_path):
    categories = sorted([
        folder for folder in os.listdir(dataset_path)
        if os.path.isdir(os.path.join(dataset_path, folder))
    ])
    return {category: idx for idx, category in enumerate(categories)}



In [7]:
dataset = "data/iphone"
categories = get_category_dict(dataset)
num_images = 500
image_size = (256, 256) 
print(categories)

{'animals': 0, 'name_board': 1, 'other_vehicle': 2, 'pedestrians': 3, 'potholes': 4, 'road_sign': 5, 'speed_breaker': 6}


In [8]:
ksize = 31
sigma = 4.0
lambd = 10.0
gamma = 0.5
psi = 0
orientations = [0, np.pi/4, np.pi/2, 3*np.pi/4]

In [15]:
def get_image_paths(dataset_dir):
    all_image_paths = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                full_path = os.path.join(root, file)
                all_image_paths.append(full_path)
    return all_image_paths

In [16]:
image_paths = get_image_paths(dataset)[:num_images]

In [17]:
for path in image_paths[:5]:
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Failed to load: {path}")
    else:
        print(f"Loaded {path} shape: {img.shape}")


print(type(image_paths))
print(len(image_paths))
print(image_paths[:5])

Loaded data/iphone\animals\10AM-2PM\IMG_3546.jpg shape: (3024, 4032)
Loaded data/iphone\animals\10AM-2PM\IMG_3547.jpg shape: (3024, 4032)
Loaded data/iphone\animals\10AM-2PM\IMG_3552.jpg shape: (3024, 4032)
Loaded data/iphone\animals\10AM-2PM\IMG_3595.jpg shape: (3024, 4032)
Loaded data/iphone\animals\10AM-2PM\IMG_3596.jpg shape: (3024, 4032)
<class 'list'>
459
['data/iphone\\animals\\10AM-2PM\\IMG_3546.jpg', 'data/iphone\\animals\\10AM-2PM\\IMG_3547.jpg', 'data/iphone\\animals\\10AM-2PM\\IMG_3552.jpg', 'data/iphone\\animals\\10AM-2PM\\IMG_3595.jpg', 'data/iphone\\animals\\10AM-2PM\\IMG_3596.jpg']


In [18]:
gabor_features = []

In [19]:
for path in tqdm(image_paths, desc="Applying Gabor Filters"):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    img = cv2.resize(img, image_size)
    filtered_stack = []

    for theta in orientations:
        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        filtered = cv2.filter2D(img, cv2.CV_32F, kernel)
        filtered_stack.append(filtered)

    gabor_feature = np.stack(filtered_stack, axis=-1)
    gabor_features.append(gabor_feature)

gabor_features = np.array(gabor_features)

Applying Gabor Filters: 100%|██████████| 459/459 [00:39<00:00, 11.51it/s]


In [20]:
print(gabor_features)

[[[[ 1.62329126e+03  1.88491284e+03  1.66007581e+03  1.88491284e+03]
   [ 1.62921008e+03  1.88446265e+03  1.66028613e+03  1.88446265e+03]
   [ 1.64599634e+03  1.88317944e+03  1.66017480e+03  1.88317944e+03]
   ...
   [ 1.91115637e+03  2.13318652e+03  1.98159924e+03  2.13318652e+03]
   [ 1.95585474e+03  2.13403369e+03  1.98356750e+03  2.13403345e+03]
   [ 1.97348218e+03  2.13428223e+03  1.98417651e+03  2.13428223e+03]]

  [[ 1.62636035e+03  1.88502222e+03  1.65854419e+03  1.88502246e+03]
   [ 1.63180566e+03  1.88397778e+03  1.65880164e+03  1.88535217e+03]
   [ 1.64739209e+03  1.88257251e+03  1.65879260e+03  1.88460559e+03]
   ...
   [ 1.91071997e+03  2.13142529e+03  1.96622534e+03  2.13189160e+03]
   [ 1.95531873e+03  2.13267041e+03  1.96801318e+03  2.13350903e+03]
   [ 1.97290820e+03  2.13356152e+03  1.96858838e+03  2.13356177e+03]]

  [[ 1.63711841e+03  1.88559326e+03  1.65501758e+03  1.88559314e+03]
   [ 1.64083716e+03  1.88418176e+03  1.65535400e+03  1.88661597e+03]
   [ 1.65209485e

In [21]:
def nearest_power_of_2(n):
    return 2 ** int(np.ceil(np.log2(n)))

In [22]:
def reduce_gabor_features(gabor_features, out_dim=4):
    reduced_features = []
    for feat in gabor_features:
        pooled = np.mean(feat, axis=(0, 1)) 
        normalized = np.interp(pooled, (pooled.min(), pooled.max()), (0, np.pi))
        reduced_features.append(normalized[:out_dim])
    return np.array(reduced_features)

In [23]:
X_quantum = reduce_gabor_features(gabor_features, out_dim=9)
print(X_quantum)
print("")
print(X_quantum.shape)
num_qubits = X_quantum.shape[1]

[[0.         3.14159265 0.01155662 3.13506189]
 [0.00329538 3.12694063 0.         3.14159265]
 [0.         3.14137573 0.00515474 3.14159265]
 ...
 [0.01368113 3.10408402 0.         3.14159265]
 [0.00794866 3.11412743 0.         3.14159265]
 [0.01114428 3.14159265 0.         3.13782152]]

(459, 4)


In [24]:
X_flat = X_quantum.flatten() 
print(X_flat)

[0.         3.14159265 0.01155662 ... 3.14159265 0.         3.13782152]


In [27]:
def pad_to_power_of_two(arr):
    length = len(arr)
    target_length = 2**int(np.ceil(np.log2(length)))
    padded = np.zeros(target_length)
    padded[:length] = arr
    return padded

X_padded = pad_to_power_of_two(X_flat)
print(X_padded)

[0.         3.14159265 0.01155662 ... 0.         0.         0.        ]


In [28]:
X_norm = X_padded / np.linalg.norm(X_padded)
print(X_norm)

[0.         0.03321052 0.00012217 ... 0.         0.         0.        ]


In [30]:
num_qubits = int(np.log2(len(X_norm)))
dev = qml.device("default.qubit", wires=num_qubits)
print(num_qubits)

11


In [ ]:
def asetez_layer():
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
        qml.RY(np.pi / 4, wires=i)

    for i in range(num_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

In [32]:
def get_qubo_hamiltonian(num_qubits):
    np.random.seed(42)
    coeffs = []
    obs = []

    for i in range(num_qubits):
        coeffs.append(np.random.uniform(-1, 1))
        obs.append(qml.PauliZ(i))

    for i in range(num_qubits - 1):
        coeffs.append(np.random.uniform(-1, 1))
        obs.append(qml.PauliZ(i) @ qml.PauliZ(i + 1))

    return qml.Hamiltonian(coeffs, obs)

cost_h = get_qubo_hamiltonian(num_qubits)
print(cost_h)

-0.250919762305275 * Z(0) + 0.9014286128198323 * Z(1) + 0.4639878836228102 * Z(2) + 0.1973169683940732 * Z(3) + -0.687962719115127 * Z(4) + -0.6880109593275947 * Z(5) + -0.8838327756636011 * Z(6) + 0.7323522915498704 * Z(7) + 0.2022300234864176 * Z(8) + 0.416145155592091 * Z(9) + -0.9588310114083951 * Z(10) + 0.9398197043239886 * (Z(0) @ Z(1)) + 0.6648852816008435 * (Z(1) @ Z(2)) + -0.5753217786434477 * (Z(2) @ Z(3)) + -0.6363500655857988 * (Z(3) @ Z(4)) + -0.6331909802931324 * (Z(4) @ Z(5)) + -0.39151551408092455 * (Z(5) @ Z(6)) + 0.04951286326447568 * (Z(6) @ Z(7)) + -0.13610996271576847 * (Z(7) @ Z(8)) + -0.4175417196039162 * (Z(8) @ Z(9)) + 0.22370578944475894 * (Z(9) @ Z(10))


In [33]:
def qaoa_layer(gamma, beta, cost_h):
    qml.templates.ApproxTimeEvolution(cost_h, gamma, 1)
    for i in range(num_qubits):
        qml.RX(2 * beta, wires=i)

In [38]:
@qml.qnode(dev)
def qaoa_amplitude_circuit(params):
    gamma, beta = params

    qml.AmplitudeEmbedding(X_norm, wires=range(num_qubits), normalize=False)
    
    for _ in range(3):
        asetez_layer()

    qaoa_layer(gamma, beta, cost_h)

    return qml.expval(cost_h)
sample_params = (0.1, 0.2)
result = qaoa_amplitude_circuit(sample_params)
print(result)

-0.5578912869480913


c:\Users\satya\AppData\Local\Programs\Python\Python313\Lib\site-packages\pennylane\ops\op_math\composite.py:209: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


In [39]:
def objective(params):
    return qaoa_amplitude_circuit(params)

init_params = np.array([0.1, 0.1], requires_grad=True)

opt_result = minimize(objective, init_params, method="COBYLA")

print("Optimal gamma, beta:", opt_result.x)
print("Minimized expected cost:", opt_result.fun)

Optimal gamma, beta: [ 0.53852863 -0.3789314 ]
Minimized expected cost: -2.191958718692565


In [40]:
@qml.qnode(dev)
def qaoa_final_state(params):
    gamma, beta = params
    qml.AmplitudeEmbedding(X_norm, wires=range(num_qubits), normalize=False)
    
    for _ in range(3):
        asetez_layer()

    qaoa_layer(gamma, beta, cost_h)

    return qml.state()


final_state = qaoa_final_state(opt_result.x)
print("Final state shape:", final_state.shape)
print(final_state)

Final state shape: (2048,)
[ 0.10354685+0.01388517j -0.08343092+0.0715744j  -0.084586  +0.02077485j
 ...  0.00647067-0.03534645j  0.00455149-0.00938481j
 -0.0108    -0.0006851j ]
